In [3]:
import pandas as pd
import numpy as np
import keras
import glob
import matplotlib.pyplot as plt
import scipy
from sklearn.metrics import roc_curve, auc
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, GlobalAveragePooling2D, Input, BatchNormalization, Multiply, Activation
from keras.optimizers import RMSprop, SGD
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from keras import backend as K
import os
np.random.seed(2020)

In [23]:
#train files 전처리
train_datagen = ImageDataGenerator(featurewise_center=False,
                 samplewise_center=False,
                 featurewise_std_normalization=False,
                 samplewise_std_normalization=False,
                 zca_whitening=False,
                 rotation_range=5,
                 width_shift_range=0.05,
                 height_shift_range=0.05,
                # shear_range=0.2,
                 zoom_range=0.2,
                 #channel_shift_range=0.,
                #fill_mode='nearest',
                 #cval=0.,
                 horizontal_flip=True,
                 vertical_flip=False,
                 rescale=1/255,  #rescale to [0-1], add zoom range of 0.2x and horizontal flip
                 validation_split=0.2) # set validation split)

train_generator = train_datagen.flow_from_directory(
        "food-101\images",
        target_size=(224,224),
        batch_size=64,
        subset='training') # set as training data

#validation set 생성
validation_generator = train_datagen.flow_from_directory(
    "food-101\images",
    target_size=(224,224),
    batch_size=64,
    subset='validation') # set as validation data

Found 45600 images belonging to 57 classes.
Found 11400 images belonging to 57 classes.


In [24]:
#Modeling
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224,224,3)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(57, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [25]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 220, 220, 32)      9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 110, 110, 32)      0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 110, 110, 32)      0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 108, 108, 64)      18496     
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 106, 106, 64)      36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 53, 53, 64)        0         
__________

In [27]:
#Model 학습 (using deeplearning)
#steps_per_epoch이 작을수록 드는 시간이 적음
history=model.fit_generator(
        train_generator,
        steps_per_epoch=100,
        epochs=100,
        validation_data=validation_generator,
        validation_steps=50)

Epoch 1/100
100/100 [==============================] - 147s 1s/step - loss: 4.0484 - acc: 0.0213 - val_loss: 4.0394 - val_acc: 0.0194
Epoch 2/100
100/100 [==============================] - 137s 1s/step - loss: 4.0375 - acc: 0.0222 - val_loss: 4.0343 - val_acc: 0.0256
Epoch 3/100
100/100 [==============================] - 137s 1s/step - loss: 4.0285 - acc: 0.0252 - val_loss: 4.0249 - val_acc: 0.0300
Epoch 4/100
100/100 [==============================] - 133s 1s/step - loss: 4.0135 - acc: 0.0284 - val_loss: 4.0074 - val_acc: 0.0309
Epoch 5/100
100/100 [==============================] - 134s 1s/step - loss: 4.0045 - acc: 0.0317 - val_loss: 3.9935 - val_acc: 0.0391
Epoch 6/100
100/100 [==============================] - 133s 1s/step - loss: 3.9872 - acc: 0.0348 - val_loss: 3.9750 - val_acc: 0.0413
Epoch 7/100
100/100 [==============================] - 133s 1s/step - loss: 3.9791 - acc: 0.0383 - val_loss: 3.9644 - val_acc: 0.0506
Epoch 8/100
100/100 [==============================] - 130s 1s

Epoch 62/100
100/100 [==============================] - 128s 1s/step - loss: 3.4202 - acc: 0.1428 - val_loss: 3.3807 - val_acc: 0.1616
Epoch 63/100
100/100 [==============================] - 128s 1s/step - loss: 3.4343 - acc: 0.1461 - val_loss: 3.3725 - val_acc: 0.1628
Epoch 64/100
100/100 [==============================] - 131s 1s/step - loss: 3.4305 - acc: 0.1462 - val_loss: 3.3718 - val_acc: 0.1694
Epoch 65/100
100/100 [==============================] - 129s 1s/step - loss: 3.4037 - acc: 0.1448 - val_loss: 3.3820 - val_acc: 0.1616
Epoch 66/100
100/100 [==============================] - 130s 1s/step - loss: 3.4048 - acc: 0.1520 - val_loss: 3.3574 - val_acc: 0.1691
Epoch 67/100
100/100 [==============================] - 129s 1s/step - loss: 3.4122 - acc: 0.1431 - val_loss: 3.3204 - val_acc: 0.1772
Epoch 68/100
100/100 [==============================] - 128s 1s/step - loss: 3.3933 - acc: 0.1478 - val_loss: 3.3216 - val_acc: 0.1877
Epoch 69/100
100/100 [==============================] -

In [28]:
# 모델 평가하기
print("-- Evaluate --")

scores = model.evaluate_generator(
            validation_generator, 
            steps = 5)

print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

-- Evaluate --
acc: 18.44%


In [29]:
from keras.models import load_model

model.save('food_classfication.h5')